In [1]:
import torch
from torch.distributions import Uniform as t_unif

/Users/siddarthchilukuri/Documents/GitHub/teaching-myself-AI/.venv/lib/python3.12/site-packages/torch/_subclasses/functional_tensor.py:295: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_numpy.cpp:84.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


In [222]:
init_pop = torch.tensor([[1,1,1,1,1,1,1,1], 
                         [2,2,2,2,2,2,2,2], 
                         [3,3,3,3,3,3,3,3], 
                         [4,4,4,4,4,4,4,4], 
                         [1,2,3,4,1,2,3,4], 
                         [4,3,2,1,4,3,2,1], 
                         [1,2,1,2,1,2,1,2], 
                         [3,4,3,4,3,4,3,4]]) 

In [223]:
def n_satisfied(state):
    """Returns the number of satisfied constraints

    Params:
    state: array-like, shape=[n_variables]
        Array of variable assignments in the format [A, B, C, D, E, F, G, H]

    Returns:
    int:
        Number of satisfied constraints
    """
    n_s = 0
    A, B, C, D, E, F, G, H = (var for var in state)
    
    if (A > G):
        n_s += 1
    if (A <= H):
        n_s += 1
    if (abs(F - B) == 1):
        n_s += 1
    if (G < H):
        n_s += 1
    if (abs(G - C) == 1):
        n_s += 1
    if ((H - C) % 2 == 0):
        n_s += 1
    if (H != D):
        n_s += 1
    if (D >= G):
        n_s += 1
    if (D != C):
        n_s += 1
    if (E != C):
        n_s += 1
    if (E < D - 1):
        n_s += 1
    if (E != H - 2):
        n_s += 1
    if (G != F):
        n_s += 1
    if (H != F):
        n_s += 1
    if (C != F):
        n_s += 1
    if (D != F - 1):
        n_s += 1
    if (abs(E - F) % 2 == 1):
        n_s += 1
        
    return n_s

In [224]:
n_satisfied(init_pop[4])

12

In [225]:
def get_survival_ranges(population, fitness_fn, print_progress):
    """Returns an array with each state given a range of floats with 
    length proportional to its fitness score which can later be sampled 

    Params:
    population: {array-like} (matrix), shape = [n_states, n_variables]]
        A matrix of states, each state being an array of variable assignments
    fitness_fn: function 
        Fitness function used to evaluate the states
    print_progress: boolean
        True if user wants the algorithm to print progress

    Returns:
    array: 
        array of ranges with the i-th variable having range [arr[i - 1], arr[i]] 
        (arr[i - 1] is 0 when i = 0)
    """
    s_arr = []
    
    for i in range(len(population)):
        n_s = fitness_fn(population[i])
        s_arr.append(n_s)
        
    s = sum(s_arr)    
    for i in range(len(s_arr)):
        print(f"State {i + 1}: {population[i]}\n" +
              f"         Fitness score: {s_arr[i]}, " + 
              f"Parent likelihood: " + 
              f"{torch.math.floor((s_arr[i] / s) * 10000)/100}%") if print_progress else 0
        s_arr[i] /= s
        if (i != 0):
            s_arr[i] += s_arr[i - 1]        
    
    return s_arr

In [226]:
sr = get_survival_ranges(init_pop, n_satisfied, True)

State 1: tensor([1, 1, 1, 1, 1, 1, 1, 1])
         Fitness score: 5, Parent likelihood: 8.47%
State 2: tensor([2, 2, 2, 2, 2, 2, 2, 2])
         Fitness score: 5, Parent likelihood: 8.47%
State 3: tensor([3, 3, 3, 3, 3, 3, 3, 3])
         Fitness score: 5, Parent likelihood: 8.47%
State 4: tensor([4, 4, 4, 4, 4, 4, 4, 4])
         Fitness score: 5, Parent likelihood: 8.47%
State 5: tensor([1, 2, 3, 4, 1, 2, 3, 4])
         Fitness score: 12, Parent likelihood: 20.33%
State 6: tensor([4, 3, 2, 1, 4, 3, 2, 1])
         Fitness score: 9, Parent likelihood: 15.25%
State 7: tensor([1, 2, 1, 2, 1, 2, 1, 2])
         Fitness score: 9, Parent likelihood: 15.25%
State 8: tensor([3, 4, 3, 4, 3, 4, 3, 4])
         Fitness score: 9, Parent likelihood: 15.25%


In [227]:
def select_parents_crossover(survival_range, num_pairs):
    """Select which parents to cross and a crossover point 
    based off of samples from a uniform probability distribution 

    Params:
    survival_range: array-like, shape=[n_ranges]
        Array of ranges corresponding to each state's survival
    num_pairs: int
        Number of parent pairs to generate

    Returns:
    {array-like} (matrix), shape = [n_pairs, 3]: 
        2-dimensional array with each entry in the format 
        [parent1, parent2, crossover_point]
    """
    parent_pairs = []
    
    for i in range(num_pairs):
        p1 = t_unif(0, 1).sample()
        p2 = t_unif(0, 1).sample()

        for j in range(len(survival_range)):
            if (p1 < survival_range[j]):
                p1 = j
                break

        for j in range(len(survival_range)):
            if (p2 < survival_range[j]):
                p2 = j
                break
            
        mutation_var = int(t_unif(0, len(survival_range) - 1).sample().item())

        parent_pairs.append([p1, p2, mutation_var])
    
    return parent_pairs

In [228]:
select_parents_crossover(sr, 4)

[[7, 7, 4], [1, 5, 1], [4, 5, 0], [2, 0, 0]]

In [229]:
def genetic(init_pop, domain, fitness_fn, num_iters, max_satisfy, print_progress = False):
    """Genetic algorithm to find solutions of CSPs

    Params:
    init_pop: {array-like} (matrix), shape = [n_states, n_variables]
        2-dimensional array of states, each state being an array of variable assignments
    var_domain: array-like, shape = [n_domain]
        Domain of the variables
    fitness_fn: function
        Fitness function used to evaluate states
    num_iters: int
        Number of iterations to run the algorithm for
    max_c_satisfy: int
        Maximum number of constraints that can be satisfied (goal)
    print_progress: boolean (optional)
        True if user wants the algorithm to print progress, False
        by default

    Returns:
    {array-like} (matrix), shape = [n_states, n_variables]: 
        2-dimensional array of states after running the algorithm for 
        min(n_iterations, n_iterations till solution)
    """
    pop_ = init_pop
    
    # Lookup table to convert indices to variable names
    int_to_var = {0: "A", 1: "B", 2: "C", 3: "D", 4: "E", 5: "F", 6: "G", 7: "H"}
    
    # Genetic algorithm for-loop
    for i in range(num_iters):
        if print_progress:
            print(f"Generation {i}:")
            for i in range(len(pop_)):
                print(f"State {i + 1}: {pop_[i]}")
        
        # Getting fitness scores and corresponding survival ranges
        sr = get_survival_ranges(pop_, fitness_fn, print_progress)
        
        # Getting 4 parent pairs and a random crossover point for each pair (Selection)
        parent_pairs = select_parents_crossover(sr, int(len(pop_) / 2))
        
        # Initializing an empty 2D array for the next generation
        new_gen = []
        
        print("\nCrossing") if print_progress else 0
        # Performing crossing for each of the parent pairs about the crossover points (Crossing)
        for j in range(len(parent_pairs)):            
            parent_pair = parent_pairs[j]
            
            print(f"Pairing {j + 1}: " +
                  f"State {parent_pairs[j][0] + 1} x State {parent_pairs[j][1] + 1} " +
                  f"with crossover point {parent_pairs[j][2]}") if print_progress else 0
            
            c1 = pop_[parent_pair[0]]
            c2 = pop_[parent_pair[1]]
            crossover_point = parent_pair[2]
            
            temp = c1[crossover_point: ]
            
            c1[crossover_point: ] = c2[crossover_point: ]
            c2[crossover_point: ] = temp
            
            print(f"Children: {c1} \n" + 
                  f"          {c2}") if print_progress else 0
            
            new_gen.append(c1)
            new_gen.append(c2)
         
        # Performing mutation with a 30% chance of a variable in a state being mutated (Mutation)
        for j in range(len(new_gen)):
            curr_c = new_gen[j]
            
            if (t_unif(0, 1).sample() <= 0.3):
                mutation_var_i = int(t_unif(0, len(c1) - 1).sample().item())
                mutated_var = domain[int(t_unif(0, len(domain) - 1).sample().item())]
                
                curr_c[mutation_var_i] = mutated_var
                
                print(f"Child {j + 1} :"+
                      f"Variable {int_to_var[mutation_var_i]} "+ 
                      f"(index {mutation_var_i}) "+
                      f"mutated to {curr_c[mutation_var_i]}\n"+
                      f"        {curr_c}") if print_progress else 0
            else:
                print(f"Child {j + 1}: No mutation") if print_progress else 0
        
        if print_progress:
            print(f"\nNext generation")
            for j in range(len(new_gen)):
                print(f"State {j + 1}: {new_gen[j]}")
            print("\n" + "=" * 51 + "\n")
            
        # Updating population to be the new generation
        pop_ = new_gen
        print(pop_)
        
        # Checking to see if any state satisfies all constraints
        for i in range(len(pop_)):
            if (fitness_fn(pop_[i]) == max_satisfy):
                print(f"Found solution: {pop_[i]} in {i + 1} iterations")
                return pop_
    
    print(f"Population after {num_iters} iterations: ")
    for i in range(len(pop_)):
                print(f"State {i + 1}: {pop_[i]}")
    return pop_

    

In [230]:
genetic(init_pop, [1, 2, 3, 4], n_satisfied, 100, 17)

[tensor([3, 2, 2, 2, 2, 2, 2, 2]), tensor([2, 2, 2, 2, 2, 2, 2, 2]), tensor([3, 2, 2, 2, 2, 2, 2, 2]), tensor([3, 2, 2, 2, 2, 2, 2, 2]), tensor([4, 3, 3, 1, 4, 3, 2, 1]), tensor([4, 3, 3, 1, 4, 3, 2, 1]), tensor([1, 2, 3, 4, 1, 2, 4, 4]), tensor([4, 2, 4, 4, 4, 4, 4, 4])]
[tensor([3, 2, 2, 3, 2, 2, 2, 2]), tensor([3, 2, 2, 3, 2, 2, 2, 2]), tensor([2, 2, 2, 2, 3, 2, 4, 4]), tensor([2, 2, 3, 4, 1, 1, 4, 4]), tensor([4, 3, 3, 1, 4, 3, 2, 1]), tensor([4, 3, 3, 1, 4, 3, 2, 1]), tensor([2, 2, 3, 4, 1, 1, 4, 4]), tensor([2, 2, 3, 4, 1, 1, 4, 4])]
[tensor([2, 1, 2, 3, 2, 2, 2, 2]), tensor([1, 2, 2, 2, 3, 2, 4, 4]), tensor([1, 2, 2, 2, 3, 2, 4, 4]), tensor([2, 1, 2, 3, 2, 2, 2, 2]), tensor([1, 2, 2, 2, 3, 2, 4, 4]), tensor([2, 2, 2, 2, 3, 2, 4, 4]), tensor([2, 1, 2, 3, 2, 2, 2, 2]), tensor([2, 1, 2, 3, 2, 2, 2, 2])]
[tensor([2, 1, 2, 1, 2, 2, 2, 2]), tensor([2, 1, 2, 1, 2, 2, 2, 2]), tensor([1, 2, 2, 3, 2, 2, 2, 2]), tensor([2, 1, 2, 1, 2, 2, 2, 2]), tensor([2, 1, 2, 1, 2, 2, 2, 2]), tensor([2,

[tensor([2, 2, 2, 3, 1, 1, 2, 2]),
 tensor([2, 2, 2, 3, 1, 1, 2, 2]),
 tensor([2, 2, 2, 3, 1, 1, 2, 2]),
 tensor([2, 2, 2, 3, 1, 1, 2, 2]),
 tensor([2, 2, 2, 3, 1, 1, 2, 2]),
 tensor([2, 2, 2, 3, 1, 1, 2, 2]),
 tensor([2, 2, 2, 3, 1, 1, 2, 2]),
 tensor([2, 2, 2, 3, 1, 1, 2, 2])]